<h1> Install pyspark library </h1>

In [1]:
!pip install pyspark

In [2]:
#import pyspark library 
import pyspark

In [3]:
#import spark session library
from pyspark.sql import SparkSession

In [4]:
#create SparkSession object 
spark = SparkSession \
            .builder \
            .appName("Decision_Tree_Classification")\
            .getOrCreate()

<h1> Create DataFrame </h1>

In [5]:
data1 = spark.read.format("csv").option("header","true")\
.option("inferSchema","true").load("Social_Network_Ads.csv")

In [6]:
data1.show()

+--------+------+---+---------------+---------+
| User ID|Gender|Age|EstimatedSalary|Purchased|
+--------+------+---+---------------+---------+
|15624510|  Male| 19|          19000|        0|
|15810944|  Male| 35|          20000|        0|
|15668575|Female| 26|          43000|        0|
|15603246|Female| 27|          57000|        0|
|15804002|  Male| 19|          76000|        0|
|15728773|  Male| 27|          58000|        0|
|15598044|Female| 27|          84000|        0|
|15694829|Female| 32|         150000|        1|
|15600575|  Male| 25|          33000|        0|
|15727311|Female| 35|          65000|        0|
|15570769|Female| 26|          80000|        0|
|15606274|Female| 26|          52000|        0|
|15746139|  Male| 20|          86000|        0|
|15704987|  Male| 32|          18000|        0|
|15628972|  Male| 18|          82000|        0|
|15697686|  Male| 29|          80000|        0|
|15733883|  Male| 47|          25000|        1|
|15617482|  Male| 45|          26000|   

<h1> Vectorize feature </h1>

In [7]:
from pyspark.ml.feature import * 

In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["Age","EstimatedSalary"])
vectorizer.setOutputCol("features")

data = vectorizer.transform(data1)
data.show(10, False)

+--------+------+---+---------------+---------+---------------+
|User ID |Gender|Age|EstimatedSalary|Purchased|features       |
+--------+------+---+---------------+---------+---------------+
|15624510|Male  |19 |19000          |0        |[19.0,19000.0] |
|15810944|Male  |35 |20000          |0        |[35.0,20000.0] |
|15668575|Female|26 |43000          |0        |[26.0,43000.0] |
|15603246|Female|27 |57000          |0        |[27.0,57000.0] |
|15804002|Male  |19 |76000          |0        |[19.0,76000.0] |
|15728773|Male  |27 |58000          |0        |[27.0,58000.0] |
|15598044|Female|27 |84000          |0        |[27.0,84000.0] |
|15694829|Female|32 |150000         |1        |[32.0,150000.0]|
|15600575|Male  |25 |33000          |0        |[25.0,33000.0] |
|15727311|Female|35 |65000          |0        |[35.0,65000.0] |
+--------+------+---+---------------+---------+---------------+
only showing top 10 rows



<h1> Data scalling </h1>

In [11]:
#import scaler library
from pyspark.ml.feature import MinMaxScaler

In [13]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

In [14]:
scalerModel = scaler.fit(data)

In [15]:
scaledData = scalerModel.transform(data)

In [16]:
scaledData.show()

+--------+------+---+---------------+---------+---------------+--------------------+
| User ID|Gender|Age|EstimatedSalary|Purchased|       features|      scaledFeatures|
+--------+------+---+---------------+---------+---------------+--------------------+
|15624510|  Male| 19|          19000|        0| [19.0,19000.0]|[0.02380952380952...|
|15810944|  Male| 35|          20000|        0| [35.0,20000.0]|[0.40476190476190...|
|15668575|Female| 26|          43000|        0| [26.0,43000.0]|[0.19047619047619...|
|15603246|Female| 27|          57000|        0| [27.0,57000.0]|[0.21428571428571...|
|15804002|  Male| 19|          76000|        0| [19.0,76000.0]|[0.02380952380952...|
|15728773|  Male| 27|          58000|        0| [27.0,58000.0]|[0.21428571428571...|
|15598044|Female| 27|          84000|        0| [27.0,84000.0]|[0.21428571428571...|
|15694829|Female| 32|         150000|        1|[32.0,150000.0]|[0.33333333333333...|
|15600575|  Male| 25|          33000|        0| [25.0,33000.0]|[0

In [17]:
scaledData.select("scaledFeatures").take(2)

[Row(scaledFeatures=DenseVector([0.0238, 0.0296])),
 Row(scaledFeatures=DenseVector([0.4048, 0.037]))]

<h1> without scaming use features column as a input column </h1>
<h1> with scaling use scaledFeatures column as a input column </h1>
<h1> Model Training </h1>

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [20]:
#Index Labels, adding metadata to the label column
#Fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol='Purchased', outputCol="indexedLabel").fit(scaledData)

In [21]:
#Automatically identify categorial features and index them
#We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer=\
VectorIndexer(inputCol="scaledFeatures", outputCol="indexedFeatures", maxCategories=4).fit(scaledData)

In [22]:
#split the data into training and test sets (30% held out for testing)
(trainingData, testData) = scaledData.randomSplit([0.7,0.3])

In [23]:
#Train a DecisionTree model
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [24]:
#Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [25]:
#Train model. This also runs the indexers
model = pipeline.fit(trainingData)

<h1> Model Testing </h1>

In [26]:
#Make predictions 
predictions = model.transform(testData)

In [28]:
#Select rows to display
predictions.select('prediction', 'Purchased', 'scaledFeatures').show(20)

+----------+---------+--------------------+
|prediction|Purchased|      scaledFeatures|
+----------+---------+--------------------+
|       0.0|        0|[0.07142857142857...|
|       0.0|        1|[0.52380952380952...|
|       1.0|        1|[0.95238095238095...|
|       0.0|        0|[0.40476190476190...|
|       0.0|        0|[0.09523809523809...|
|       1.0|        1|[0.59523809523809...|
|       0.0|        0|[0.5,0.4592592592...|
|       0.0|        0|[0.45238095238095...|
|       0.0|        0|[0.38095238095238...|
|       1.0|        1|[0.23809523809523...|
|       0.0|        1|[0.33333333333333...|
|       0.0|        0|[0.14285714285714...|
|       1.0|        1|[0.45238095238095...|
|       0.0|        0|[0.54761904761904...|
|       1.0|        1|[0.57142857142857...|
|       1.0|        1|[0.83333333333333...|
|       1.0|        1|[0.5,0.6740740740...|
|       0.0|        0|[0.33333333333333...|
|       0.0|        0|[0.23809523809523...|
|       1.0|        1|[0.7380952

<h1> Model Summary </h1>

In [29]:
#Select (prediction true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
labelCol="indexedLabel", predictionCol="prediction", metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.103448 


In [32]:
accuracy

0.896551724137931

In [31]:
treeModel = model.stages[2]
    # summary only
print(treeModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_48d7a3498576, depth=5, numNodes=31, numClasses=2, numFeatures=2


In [34]:
predictions.groupBy('Purchased','prediction').count().show()

+---------+----------+-----+
|Purchased|prediction|count|
+---------+----------+-----+
|        1|       0.0|    6|
|        0|       0.0|   73|
|        1|       1.0|   31|
|        0|       1.0|    6|
+---------+----------+-----+



In [35]:
#Calculate the elements of the confusion matrix
TN = predictions.filter('prediction = 0 AND Purchased = prediction').count()
TP = predictions.filter('prediction = 1 AND Purchased = prediction').count()
FN = predictions.filter('prediction = 0 AND Purchased = 1').count()
FP = predictions.filter('prediction = 0 AND Purchased = 0').count()

#Accuracy measures the proportion of correct predictions 
accuracy = (TN + TP) / (TN + TP + FN + FP)
print("Model Accuracy = %g" % (accuracy))

Model Accuracy = 0.568306
